# Imports

In [132]:
#Imports
%matplotlib widget
import pandas as pd
import numpy as np
import csv
import matplotlib
import matplotlib.pyplot as plt
import scipy
import datetime
import networkx as nx
import misc
import math
import random

# Data Generation

In [2]:
#Create empty graph
G = nx.Graph()

In [3]:
G.add_nodes_from([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

G.add_edges_from([
    (0,1),
    (1,2),
    (2,3),
    (3,4),
    (4,5),
    (5,6),
    (6,7),
    (7,8),
    (8,9),
    (9,0),

    (0,2),
    (0,3),
    (0,4),
    (1,3)
])

# Common items

In [106]:
G = nx.Graph()  # The complete graph
d = 3   # Amount of levels
staticColoring = {}   # Coloring at any moment
bucketLevels = []

In [146]:
class Bucket:

  vertices: set = {}
  coloring: dict = {}

  def __init__(self, size, level):
      self.size = size
      self.level = level

  def addVertices(self, newVertices):
    self.vertices = self.vertices.union(newVertices)
    for vertex in newVertices:
        G.nodes[vertex]['bucket'] = self

  def removeVertices(self, removeVertices):
    self.vertices = self.vertices.difference(removeVertices)
    for vertex in removeVertices:
        G.nodes[vertex]['bucket'] = None


In [50]:
# Returns whether there is still an empty bucket on the requested level
# Also returns the index of this bucket, if applicable
def isEmptyBucketOnLevel(level: int) -> tuple[bool, int]:
    for j in range(0, len(bucketLevels[level])):
        bucket = bucketLevels[level][j]
        if len(bucket.level) == 0:
                return (True, j)
    return (False, -1)

In [145]:
# Empties all buckets on a level and returns the union of all vertices
# Also ensures the colorings and vertex lists in the emptied buckets are reset
def emptyLevel(level: int):
    vertices: set = {}
    for bucket in bucketLevels[level]:
        vertices = vertices.union(bucket.vertices)
        bucket.removeVertices(bucket.vertices)
        bucket.coloring = {}
    return vertices

{1, 2, 3, 4}

# Small Bucket Algorithm

In [147]:
# Creates brand new buckets with the correct sizes
def resetBuckets(G: nx.Graph):
    nx.set_node_attributes(G, None, 'bucket')   # Reset all bucket references in G to None
    nr = G.number_of_nodes()    # Number of vertices in G
    s = math.ceil(pow(nr, 1/d)) # Amount of buckets per level

    # Create d levels of s buckets each, with capacity s^i per bucket, where i the level of the bucket
    global bucketLevels
    bucketLevels = []
    for level in range(0, d):
        bucketLevel = []
        for b in range(0, s):
            bucketLevel.append(Bucket(pow(s, level), level))
        bucketLevels.append(bucketLevel)
    bucketLevels.append([Bucket(nr, d)])
    
    staticColoring = nx.coloring.greedy_color(G)
    

In [129]:
# Update bucket contents and recolor subgraphs
def updateBuckets(b: Bucket):
    b = b
    i = 0
    while i < d:
        # If there is still an empty bucket on a level, this level does not need updating
        # Simply recompute the coloring of the most recent bucket and return
        if isEmptyBucketOnLevel(i)[0]:
            b.coloring = nx.coloring.greedy_color(G.subgraph(b.vertices))
            return
        else:
            # Else, empty all level i buckets into a single level i+1 bucket, update b to point at new bucket
            vertices = emptyLevel(i)
            d = bucketLevels[i+1][isEmptyBucketOnLevel(i+1)[1]]
            d.addVertices(vertices)
    resetBuckets()
    return

In [130]:
def removeEdge(e):
    G.remove_edge(e)

def removeVertex(v):
    G.nodes[v]['buckets'].removeVertices([v])
    G.remove_node(v)

def addEdge(e):
    G.add_edge(e)
    # Select one of the endpoints at random, remove it from a bucket and add it as usual
    if bool(random.getrandbits(1)):
        v = e[0]
    else:
        v = e[1]

    G.nodes[v]['buckets'].removeVertices([v])
    addVertex(v)

def addVertex(v):
    G.add_node(v)
    bucket = bucketLevels[0][isEmptyBucketOnLevel(0)[1]]
    bucket.addVertices([v])

In [151]:
def getColoring():
    bucketColorings = []
    for bucketLevel in bucketLevels:
        for bucket in bucketLevel:
            bucketColorings.append(bucket.coloring)
    combinedColoring = misc.combineColorings(bucketColorings)   
    totalColoring = staticColoring.copy() 
    totalColoring.update(combinedColoring)
    return totalColoring      

# Big Bucket Algorithm

# Testing and Drawing

In [6]:
coloring = nx.coloring.greedy_color(G, strategy=nx.coloring.strategy_largest_first)

In [7]:
layout = nx.spring_layout(G)

In [10]:
c = misc.combineColorings([coloring, coloring2])

In [14]:
G2 = G.subgraph([0,1,2,3])
G3 = G.subgraph([4,5,6,7,8,9])